In [1]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lenguyenquocanh-vn-fptu (lenguyenquocanh-vn-fptu-fpt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Data Preparation

In [2]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-b0417886a268b83a.parquet', 'valid': 'data/valid-00000-of-00001-846411c236133ba3.parquet'}
df_train = pd.read_parquet("hf://datasets/datnth1709/VLSP2016-NER-data/" + splits["train"])
df_valid = pd.read_parquet("hf://datasets/datnth1709/VLSP2016-NER-data/" + splits["valid"])
df = pd.concat([df_train, df_valid]).reset_index(drop=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# Tạo thêm các cột khác
def join_tokens(tokens):
    text = ' '.join(tokens)
    return text

def reform_raw_text(tokens):
    text = ' '.join(tokens)
    return text.replace("_", " ")

def label(x):
  return [id_tag[int(i)] for i in x]

def replace_7_8(lst):
    return [0 if x in (7, 8) else x for x in lst]


tag_id = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}
id_tag = {0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}


df['ner_tags'] = df['ner_tags'].apply(replace_7_8)
df['text_withseg'] = df['tokens'].apply(join_tokens)
df['text_raw'] = df['tokens'].apply(reform_raw_text)
df["ner_labels"] = df.ner_tags.apply(label)
df.columns = ['tokens', 'id', 'seg_text', 'raw_text', 'labels']
df


,tokens,id,seg_text,raw_text,labels
0,"[Không_khí, thật, náo_nhiệt, .]","[0, 0, 0, 0]",Không_khí thật náo_nhiệt .,Không khí thật náo nhiệt .,"[O, O, O, O]"
1,"[Chị, Lãnh, và, Xăng, ra, đi, ,, mình, đứng, n...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Chị Lãnh và Xăng ra đi , mình đứng nhìn hai ch...","Chị Lãnh và Xăng ra đi , mình đứng nhìn hai ch...","[O, B-PER, O, B-PER, O, O, O, O, O, O, O, O, O..."
2,"[Suy_tính, mãi, ,, khóc, mãi, rồi, Phúc, lấy, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","Suy_tính mãi , khóc mãi rồi Phúc lấy ra tờ giấ...","Suy tính mãi , khóc mãi rồi Phúc lấy ra tờ giấ...","[O, O, O, O, O, O, B-PER, O, O, O, O, O, O, O,..."
3,"[Hoà, bảo, hồi, mới, qua, đâu, có, biết, nấu_n...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, ...","Hoà bảo hồi mới qua đâu có biết nấu_nướng gì ,...","Hoà bảo hồi mới qua đâu có biết nấu nướng gì ,...","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, B-..."
4,"[Nhật_ký, của, thuyền_viên, .]","[0, 0, 0, 0]",Nhật_ký của thuyền_viên .,Nhật ký của thuyền viên .,"[O, O, O, O]"
...,...,...,...,...,...
16853,"[Nghe, thấy, đã, ghê_ghê, nhưng, Nhiêu, chưa, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",Nghe thấy đã ghê_ghê nhưng Nhiêu chưa được tườ...,Nghe thấy đã ghê ghê nhưng Nhiêu chưa được tườ...,"[O, O, O, O, O, B-PER, O, O, O, O, O, O, O, O,..."
16854,"[Nhưng, mọi, chuyện, không, dừng, ở, đó, .]","[0, 0, 0, 0, 0, 0, 0, 0]",Nhưng mọi chuyện không dừng ở đó .,Nhưng mọi chuyện không dừng ở đó .,"[O, O, O, O, O, O, O, O]"
16855,"[Hoà, bảo, thời_gian, đầu, mặc_cảm, lắm, ,, ở,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Hoà bảo thời_gian đầu mặc_cảm lắm , ở trong nh...","Hoà bảo thời gian đầu mặc cảm lắm , ở trong nh...","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
16856,"[Biết_bao, người, đã, tình_nguyện, hiến_dâng, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Biết_bao người đã tình_nguyện hiến_dâng cả cuộ...,Biết bao người đã tình nguyện hiến dâng cả cuộ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [4]:
print(df['seg_text'][1])

Chị Lãnh và Xăng ra đi , mình đứng nhìn hai chị quần xắn tròn trên vế , lặn_lội qua dòng suối nước chảy rần_rần , tự_nhiên nước_mắt mình rưng_rưng ...


In [5]:
print(df['raw_text'][1])

Chị Lãnh và Xăng ra đi , mình đứng nhìn hai chị quần xắn tròn trên vế , lặn lội qua dòng suối nước chảy rần rần , tự nhiên nước mắt mình rưng rưng ...


In [6]:
print(df['id'][1])

[np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]


# Get Embedding Vectors

In [7]:
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# Load PhoBERT tokenizer và model
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModel.from_pretrained("vinai/phobert-base")
model.eval()

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [8]:
tokens_e1 = ['Tôi', 'thích', 'học', 'học@@', 'máy']


In [9]:
# Hàm gộp các embedding vectors của token bị tách ra khi qua SentencePiece
def group_embeddings(tokens, embeddings):
    word_embeddings = []
    current_vecs = []

    for token, emb in zip(tokens, embeddings):
        if token in ["<s>", "</s>"]:
            continue
        #Token là subword (có đuôi "@@")
        if token.endswith("@@"):
            current_vecs.append(emb)
        else: #Token là phần cuối của một từ (không có "@@")
            current_vecs.append(emb)
            word_emb = torch.mean(torch.stack(current_vecs), dim=0)
            word_embeddings.append(word_emb)
            current_vecs = []

    if current_vecs:  # Trong trường hợp sót lại cuối câu
        word_emb = torch.mean(torch.stack(current_vecs), dim=0)
        word_embeddings.append(word_emb)

    return word_embeddings

In [10]:
raw_e = df['raw_text'][0]
print(raw_e)

Không khí thật náo nhiệt .


In [11]:
sentence_e  = 'Chị Lãnh và Xăng ra đi , mình đứng nhìn hai chị quần xắn tròn trên vế , lặn_lội qua dòng suối nước chảy rần_rần , tự_nhiên nước_mắt mình rưng_rưng ...'
id_e = [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
def dem_so_tu(cau):
    # Tách câu thành các từ bằng khoảng trắng
    tu_danh_sach = cau.split()
    # Đếm số lượng từ
    return len(tu_danh_sach)

# Ví dụ sử dụng
sentence_e = 'Chị Lãnh và Xăng ra đi , mình đứng nhìn hai chị quần xắn tròn trên vế , lặn_lội qua dòng suối nước chảy rần_rần , tự_nhiên nước_mắt mình rưng_rưng ...'
so_tu = dem_so_tu(sentence_e)
print("Số lượng từ trong câu là:", so_tu)
input_e = tokenizer.encode(sentence_e)
tokens_e = tokenizer.convert_ids_to_tokens(input_e[0])
print(input_e)
print(len(id_e))
print('độ dài của tokens',len(input_e))
print(tokens_e)


Số lượng từ trong câu là: 31
[0, 1108, 19703, 6, 28163, 40, 57, 4, 68, 414, 364, 82, 213, 2747, 20899, 2533, 34, 23798, 4, 13468, 89, 532, 3364, 58, 2181, 33151, 4, 1124, 2396, 68, 17865, 135, 2]
31
độ dài của tokens 33
<s>


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

all_embeddings = []  # list of [seq_len_i, 768] tensors
all_labels = [] # list of [seq_len_i,] tensors
len_em = []

# count = 0

for i, row in tqdm(df.iterrows(), total=len(df)):

    # count += 1
    # if count == 500:
    #   break

    # Truy cập phần tử từng dòng
    sentence = row['seg_text']
    gold_labels = row["id"]

    # Cho sentence đi qua SentencePiece
    input_ids = tokenizer.encode(sentence, return_tensors="pt").to(device)

    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu())

    # Encode tạo embeddings
    with torch.no_grad():
        outputs = model(input_ids)
        last_hidden_state = outputs.last_hidden_state.squeeze(0).cpu()

    # Gộp các embeddings đã bị tách khi đi qua SentencePiece
    word_embeds = group_embeddings(tokens, last_hidden_state)

    # Kiểm tra số lượng embeddings và số lượng labels
    if len(word_embeds) != len(gold_labels):
        print(f"Warning: Skipping row {i} - length mismatch")
        continue

    # Thêm vào list tổng / Tới đây là data đã sẵn sàng cho training
    all_embeddings.append(torch.stack(word_embeds))
    all_labels.append(torch.tensor(gold_labels))

  0%|          | 18/16858 [00:00<08:06, 34.64it/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

  4%|▎         | 622/16858 [00:08<02:38, 102.31it/s]

  4%|▍         | 670/16858 [00:08<03:23, 79.66it/s]

  7%|▋         | 1204/16858 [00:14<02:24, 108.20it/s]

...

 90%|████████▉ | 15092/16858 [02:32<00:16, 108.92it/s]

 91%|█████████ | 15339/16858 [02:34<00:14, 104.17it/s]

 92%|█████████▏| 15451/16858 [02:35<00:13, 105.05it/s]

 92%|█████████▏| 15529/16858 [02:36<00:12, 106.31it/s]

100%|██████████| 16858/16858 [02:49<00:00, 99.47it/s] 

#Train Softmax with Pytorch

In [13]:
# from torch.utils.data import Dataset, DataLoader

# class NERDataset(Dataset):
#     def __init__(self, embeddings, labels):
#         self.embeddings = embeddings
#         self.labels = labelsinp

#     def __len__(self):
#         return len(self.embeddings)

#     def __getitem__(self, idx):
#         return self.embeddings[idx], self.labels[idx]

# def collate_fn(batch): # Hàm này được dùng để "gom" các mẫu có độ dài khác nhau vào một batch, thông qua padding.

#     embeddings, labels = zip(*batch)
#     lengths = [len(x) for x in embeddings]

#     # Padding
#     max_len = max(lengths)
#     padded_embs = torch.stack([
#         torch.cat([e, torch.zeros(max_len - len(e), e.size(1))]) for e in embeddings
#     ])
#     padded_labels = torch.stack([
#         torch.cat([l, torch.full((max_len - len(l),), -1)]) for l in labels
#     ])
#     return padded_embs, padded_labels, lengths

In [14]:
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score
# from tqdm import tqdm
# # Initialize Weights & Biases
# wandb.init(
#     project="NER",
#     name="Softmax_VLSP2016",
#     config={
#         "epochs": 20,
#         "batch_size": 16,
#         "learning_rate": 1e-3,
#         "input_dim": 768,
#         "test_size": 0.2
#     }
# )
# # --- 1. Define the Softmax Model ---
# class SoftmaxTagger(nn.Module):
#     def __init__(self, input_dim, num_tags):
#         super().__init__()
#         # Một lớp tuyến tính duy nhất ánh xạ từ chiều nhúng đến số lượng thẻ
#         self.hidden2tag = nn.Linear(input_dim, num_tags)

#     def forward(self, x):
#         # Truyền các embedding qua các lớp tuyến tính
#         emissions = self.hidden2tag(x)
#         return emissions
#     def save_model(self, path):
#         torch.save(self.state_dict(), path)

#     def load_model(self, path):
#         self.load_state_dict(torch.load(path))
#         self.eval()

In [15]:
# # --- 2. Define the Evaluation Function ---
# def evaluate_softmax(model, dataloader, loss_fn, device):
#     model.eval()  # Set the model to evaluation mode
#     total_loss = 0
#     all_preds = []
#     all_true = []

#     with torch.no_grad():
#         for x, y, lengths in dataloader:
#             x, y = x.to(device), y.to(device)

#             #Model output
#             emissions = model(x)

#             # Calculate loss, ignoring padding -1
#             # Reshape for CrossEntropyLoss: (N, C) and (N)
#             loss = loss_fn(emissions.view(-1, model.hidden2tag.out_features), y.view(-1))
#             total_loss += loss.item()

#             #Nhận dự đoán bằng cách tìm tag có điểm cao nhất
#             preds = torch.argmax(emissions, dim=2)

#             #Làm phẳng các dự đoán và nhãn thực, loại bỏ padding
#             for i in range(len(lengths)):
#                 true_len = lengths[i]
#                 all_true.extend(y[i, :true_len].cpu().numpy())
#                 all_preds.extend(preds[i, :true_len].cpu().numpy())

#     # Calculate metrics
#     precision, recall, f1, _ = precision_recall_fscore_support(
#         all_true, all_preds, average='macro', zero_division=0
#     )
#     accuracy = accuracy_score(all_true, all_preds)

#     return total_loss / len(dataloader), precision, recall, f1, accuracy


In [16]:
# # --- 3. Prepare Data and Training Setup ---

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Split data into training and testing sets
# train_embs, test_embs, train_labels, test_labels = train_test_split(
#     all_embeddings, all_labels, test_size=0.2, random_state=42
# )

# # Create Datasets and DataLoaders
# train_dataset = NERDataset(train_embs, train_labels)
# test_dataset = NERDataset(test_embs, test_labels)

# BATCH_SIZE = 32
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# # Model, Loss, and Optimizer Initialization
# INPUT_DIM = 768  # PhoBERT base embedding size
# NUM_TAGS = max(label.max().item() for label in all_labels) + 1
# LEARNING_RATE = 1e-3
# EPOCHS = 10

# model = SoftmaxTagger(INPUT_DIM, NUM_TAGS).to(device)
# # Use CrossEntropyLoss, which combines LogSoftmax and NLLLoss.
# # ignore_index=-1 tells it to skip padded positions during loss calculation.
# loss_fn = nn.CrossEntropyLoss(ignore_index=-1)
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [17]:
# # --- 4. Training Loop ---

# print("Starting Softmax Model Training...")
# for epoch in range(1, EPOCHS + 1):
#     model.train()
#     total_train_loss = 0

#     train_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
#     for x, y, lengths in train_bar:
#         x, y = x.to(device), y.to(device)

#         # Forward pass
#         emissions = model(x)

#         # Calculate loss
#         # Reshape for CrossEntropyLoss: (N, C) and (N)
#         loss = loss_fn(emissions.view(-1, NUM_TAGS), y.view(-1))

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         total_train_loss += loss.item()
#         train_bar.set_postfix(loss=total_train_loss / len(train_bar))

#     # Evaluate after each epoch
#     avg_test_loss, precision, recall, f1, accuracy = evaluate_softmax(model, test_loader, loss_fn, device)

#     print(f"Epoch {epoch}/{EPOCHS} -> Train Loss: {total_train_loss/len(train_loader):.4f} | "
#           f"Test Loss: {avg_test_loss:.4f} | test_f1: {f1:.4f} | test_acc: {accuracy:.4f}")


# print("\n--- Training Finished ---")

In [18]:
# # --- 5. Final Evaluation Report ---
# print("\nFinal Test Set Performance:")
# model.eval()
# all_preds_final, all_true_final = [], []
# with torch.no_grad():
#     for x, y, lengths in tqdm(test_loader, desc="Generating Final Report"):
#         x, y = x.to(device), y.to(device)
#         preds = torch.argmax(model(x), dim=2)
#         for i in range(len(lengths)):
#             true_len = lengths[i]
#             all_true_final.extend(y[i, :true_len].cpu().numpy())
#             all_preds_final.extend(preds[i, :true_len].cpu().numpy())

# # Generate and print the classification report
# target_names = [id_tag[i] for i in range(NUM_TAGS)]
# report = classification_report(all_true_final, all_preds_final, target_names=target_names, digits=4)
# print("\nClassification Report:\n", report)


In [19]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score
from tqdm import tqdm
import wandb

# Create checkpoint directory
os.makedirs("checkpoints", exist_ok=True)

# Initialize Weights & Biases
wandb.init(
    project="NER",
    name="Softmax_VLSP2016",
    config={
        "epochs": 20,
        "batch_size": 16,
        "learning_rate": 1e-3,
        "input_dim": 768,
        "test_size": 0.2
    }
)

# --- Dataset ---
class NERDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

def collate_fn(batch):
    embeddings, labels = zip(*batch)
    lengths = [len(x) for x in embeddings]
    max_len = max(lengths)

    padded_embs = torch.stack([
        torch.cat([e, torch.zeros(max_len - len(e), e.size(1))]) for e in embeddings
    ])
    padded_labels = torch.stack([
        torch.cat([l, torch.full((max_len - len(l),), -1)]) for l in labels
    ])
    return padded_embs, padded_labels, lengths

# --- Model ---
class SoftmaxTagger(nn.Module):
    def __init__(self, input_dim, num_tags):
        super().__init__()
        self.hidden2tag = nn.Linear(input_dim, num_tags)

    def forward(self, x):
        return self.hidden2tag(x)

    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        self.load_state_dict(torch.load(path))
        self.eval()

# --- Evaluation ---
def evaluate_softmax(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_preds, all_true = [], []

    with torch.no_grad():
        for x, y, lengths in dataloader:
            x, y = x.to(device), y.to(device)
            emissions = model(x)
            loss = loss_fn(emissions.view(-1, model.hidden2tag.out_features), y.view(-1))
            total_loss += loss.item()
            preds = torch.argmax(emissions, dim=2)
            for i in range(len(lengths)):
                true_len = lengths[i]
                all_true.extend(y[i, :true_len].cpu().numpy())
                all_preds.extend(preds[i, :true_len].cpu().numpy())

    precision, recall, f1, _ = precision_recall_fscore_support(all_true, all_preds, average='macro', zero_division=0)
    accuracy = accuracy_score(all_true, all_preds)

    return total_loss / len(dataloader), precision, recall, f1, accuracy, all_preds, all_true

# Train/test split
train_embs, test_embs, train_labels, test_labels = train_test_split(
    all_embeddings, all_labels, test_size=0.2, random_state=42
)

train_dataset = NERDataset(train_embs, train_labels)
test_dataset = NERDataset(test_embs, test_labels)

BATCH_SIZE = wandb.config.batch_size
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

INPUT_DIM = wandb.config.input_dim
NUM_TAGS = max(label.max().item() for label in all_labels) + 1
LEARNING_RATE = wandb.config.learning_rate
EPOCHS = wandb.config.epochs

model = SoftmaxTagger(INPUT_DIM, NUM_TAGS).to(device)
loss_fn = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

best_f1 = 0
best_acc = 0

# --- Training Loop ---
print("Starting Softmax Model Training...")
for epoch in range(1, EPOCHS + 1):
    model.train()
    total_train_loss = 0
    all_train_preds, all_train_true = [], []

    train_bar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
    for x, y, lengths in train_bar:
        x, y = x.to(device), y.to(device)
        emissions = model(x)
        loss = loss_fn(emissions.view(-1, NUM_TAGS), y.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
        train_bar.set_postfix(loss=total_train_loss / len(train_bar))

        preds = torch.argmax(emissions, dim=2)
        for i in range(len(lengths)):
            true_len = lengths[i]
            all_train_true.extend(y[i, :true_len].cpu().numpy())
            all_train_preds.extend(preds[i, :true_len].cpu().numpy())

    train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(
        all_train_true, all_train_preds, average='macro', zero_division=0
    )
    train_acc = accuracy_score(all_train_true, all_train_preds)

    # Validation
    val_loss, val_precision, val_recall, val_f1, val_acc, _, _ = evaluate_softmax(model, test_loader, loss_fn, device)

    # Logging to wandb
    wandb.log({
        "epoch": epoch,
        "avg_train_loss": total_train_loss / len(train_loader),
        "train_precision": train_precision,
        "train_recall": train_recall,
        "train_f1": train_f1,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_precision": val_precision,
        "val_recall": val_recall,
        "val_f1": val_f1,
        "val_acc": val_acc,
    })

    # Save best model
    if val_f1 > best_f1 or val_acc > best_acc:
        best_f1 = max(val_f1, best_f1)
        best_acc = max(val_acc, best_acc)
        ckpt_path = f"checkpoints/best_epoch_{epoch}.pt"
        model.save_model(ckpt_path)
        wandb.save(ckpt_path)
        print(f"Saved improved model to {ckpt_path}")

    print(f"Epoch {epoch}/{EPOCHS} -> Train Loss: {total_train_loss/len(train_loader):.4f} | "
          f"Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f} | Val Acc: {val_acc:.4f}")

print("\n--- Training Finished ---")

# --- Final Evaluation Report ---
print("\nFinal Test Set Performance:")
model.eval()
_, _, _, _, _, all_preds_final, all_true_final = evaluate_softmax(model, test_loader, loss_fn, device)

# Classification report table
target_names = [id_tag[i] for i in range(NUM_TAGS)]
report = classification_report(all_true_final, all_preds_final, target_names=target_names, digits=4, output_dict=True)

# Log report as wandb table
table = wandb.Table(columns=["Label", "Precision", "Recall", "F1-score", "Support"])
for label in target_names:
    row = report[label]
    table.add_data(label, row["precision"], row["recall"], row["f1-score"], row["support"])

wandb.log({"Test Classification Report": table})
print("\nClassification Report:\n", classification_report(all_true_final, all_preds_final, target_names=target_names, digits=4))


Starting Softmax Model Training...


Epoch 1/20: 100%|██████████| 841/841 [00:05<00:00, 149.09it/s, loss=0.122]


Saved improved model to checkpoints/best_epoch_1.pt
Epoch 1/20 -> Train Loss: 0.1224 | Val Loss: 0.0489 | Val F1: 0.7949 | Val Acc: 0.9847


Epoch 2/20: 100%|██████████| 841/841 [00:04<00:00, 173.24it/s, loss=0.0419]


Saved improved model to checkpoints/best_epoch_2.pt
Epoch 2/20 -> Train Loss: 0.0419 | Val Loss: 0.0359 | Val F1: 0.8518 | Val Acc: 0.9885


...

Epoch 18/20 -> Train Loss: 0.0192 | Val Loss: 0.0250 | Val F1: 0.8893 | Val Acc: 0.9914


Epoch 19/20: 100%|██████████| 841/841 [00:05<00:00, 163.39it/s, loss=0.0188]


Saved improved model to checkpoints/best_epoch_19.pt
Epoch 19/20 -> Train Loss: 0.0188 | Val Loss: 0.0253 | Val F1: 0.8926 | Val Acc: 0.9915


Epoch 20/20: 100%|██████████| 841/841 [00:04<00:00, 168.43it/s, loss=0.0188]


Saved improved model to checkpoints/best_epoch_20.pt
Epoch 20/20 -> Train Loss: 0.0188 | Val Loss: 0.0249 | Val F1: 0.8936 | Val Acc: 0.9918

--- Training Finished ---

Final Test Set Performance:

Classification Report:
               precision    recall  f1-score   support

           O     0.9973    0.9973    0.9973     68476
       B-PER     0.9869    0.9768    0.9818      1464
       I-PER     0.9810    0.9767    0.9788       686
       B-ORG     0.7709    0.8249    0.7970       257
       I-ORG     0.7981    0.7721    0.7849       430
       B-LOC     0.8809    0.9001    0.8904      1241
       I-LOC     0.8339    0.8159    0.8248       554

    accuracy                         0.9918     73108
   macro avg     0.8927    0.8948    0.8936     73108
weighted avg     0.9918    0.9918    0.9918     73108



# Lưu data

In [20]:
def save_tensors(all_embeddings, all_labels, embed_path='embeddings.pt', label_path='labels.pt'):
    torch.save(all_embeddings, embed_path)
    torch.save(all_labels, label_path)
    print(f"Saved embeddings to {embed_path} and labels to {label_path}")

In [22]:
from google.colab import drive
import shutil

# Gọi hàm đã viết
save_tensors(all_embeddings, all_labels)

# Mount và tải lên Drive
drive.mount('/content/drive')
shutil.copy('embeddings.pt', '/content/drive/My Drive')
shutil.copy('labels.pt', '/content/drive/My Drive')


Saved embeddings to embeddings.pt and labels to labels.pt
Mounted at /content/drive


'/content/drive/My Drive/labels.pt'

In [23]:
torch.save(model.state_dict(), "softmax_tagger.pth")

In [24]:
model = SoftmaxTagger(INPUT_DIM, NUM_TAGS)
model.load_state_dict(torch.load("softmax_tagger.pth"))
model.eval()  # chuyển sang chế độ đánh giá nếu cần


SoftmaxTagger(
  (hidden2tag): Linear(in_features=768, out_features=7, bias=True)
)

In [25]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model_bert = AutoModel.from_pretrained("vinai/phobert-base").to(device)  # PhoBERT để lấy embedding

# model là SoftmaxTagger đã train xong


In [26]:
def predict_ner(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()

    # Tokenize văn bản đầu vào
    input_ids = tokenizer.encode(text, return_tensors="pt").to(device)
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu())

    # Lấy embedding đầu ra từ PhoBERT
    with torch.no_grad():
        outputs = model_bert(input_ids)
        last_hidden_state = outputs.last_hidden_state.squeeze(0).cpu()

    # Gộp embedding của từ bị tách (sentencepiece)
    word_embeds = group_embeddings(tokens, last_hidden_state)

    # Chuyển sang tensor
    x_tensor = torch.stack(word_embeds).unsqueeze(0).to(device)  # (1, seq_len, 768)

    # Dự đoán
    with torch.no_grad():
        emissions = model(x_tensor)
        preds = torch.argmax(emissions, dim=2).squeeze(0).cpu().tolist()

    # Trích xuất token gốc không bị tách '@@'
    final_tokens = []
    current_token = ""
    for tok in tokens:
        if tok in ["<s>", "</s>"]:
            continue
        if tok.endswith("@@"):
            current_token += tok[:-2]
        else:
            current_token += tok
            final_tokens.append(current_token)
            current_token = ""

    # Ánh xạ sang tên nhãn
    label_names = [id_tag[i] for i in preds]

    return preds, label_names, final_tokens


In [27]:
pip install gradio

In [28]:
model = SoftmaxTagger(INPUT_DIM, NUM_TAGS) # Make sure INPUT_DIM and NUM_TAGS are defined or accessible here
model.load_state_dict(torch.load("softmax_tagger.pth"))
model.eval()  # chuyển sang chế độ đánh giá nếu cần
model.to(device) # Add this line to move the model to the device

SoftmaxTagger(
  (hidden2tag): Linear(in_features=768, out_features=7, bias=True)
)

In [29]:
import gradio as gr
import json
import tempfile
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def ner_interface(text):
    try:
        ids, labels, tokens = predict_ner(text)
        data = [[token, label, _id] for token, label, _id in zip(tokens, labels, ids)]
        json_result = {
            "tokens": tokens,
            "labels": labels,
            "label_ids": ids
        }
        return data, json_result
    except Exception as e:
        print("Error:", e)
        return [["Lỗi", str(e), ""]], {"error": str(e)}

def json_to_file(json_data):
    # Tạo file tạm thời để trả về cho gr.File tải về
    tmp = tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False, encoding='utf-8')
    json.dump(json_data, tmp, ensure_ascii=False, indent=2)
    tmp.close()
    return tmp.name


with gr.Blocks(title="Nhận dạng Thực thể (NER) với PhoBERT") as demo:
    gr.Markdown("## 📌 Hệ thống Nhận dạng Thực thể Tên (NER) sử dụng PhoBERT + Softmax")

    with gr.Row():
        with gr.Column(scale=3):
            input_text = gr.Textbox(
                lines=4,
                label="✍️ Nhập văn bản đầu vào",
                placeholder="Ví dụ: Nguyễn Văn A sinh ra ở Hà Nội."
            )
            btn = gr.Button("🚀 Nhận dạng Thực thể")

        with gr.Column(scale=5):
            output_table = gr.Dataframe(
                headers=["Token", "Label", "ID"],
                label="📄 Kết quả nhận dạng thực thể",
                wrap=True
            )
            output_json = gr.JSON(visible=False)  # Có thể bật nếu muốn hiển thị JSON

    with gr.Row():
        download_trigger = gr.Button("💾 Tải kết quả dưới dạng JSON")
        download_file = gr.File(label="📥 File JSON đã xử lý")

    # Hành động xử lý NER
    btn.click(fn=ner_interface, inputs=input_text, outputs=[output_table, output_json])
    download_trigger.click(fn=json_to_file, inputs=output_json, outputs=download_file)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c3f739dbf40a0a0681.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
